# Step-by-Step Guide: Comparing RLiG with Baselines

This notebook provides a systematic approach to comparing RLiG with baseline models. We'll go through the process step by step, making sure everything is working properly at each stage.

## Step 1: Environment Setup

First, let's make sure we have all the necessary dependencies installed.

In [3]:
# Create anaconda env with python=3.10
# Install pgmpy from dev branch: 
!pip install git+https://github.com/pgmpy/pgmpy.git@dev

  Cloning https://github.com/pgmpy/pgmpy.git (to revision dev) to /private/var/folders/13/p9hvf8t929x0626wz_slh9340000gn/T/pip-req-build-v4rrs5ju
  Running command git clone --filter=blob:none --quiet https://github.com/pgmpy/pgmpy.git /private/var/folders/13/p9hvf8t929x0626wz_slh9340000gn/T/pip-req-build-v4rrs5ju
  Resolved https://github.com/pgmpy/pgmpy.git to commit 4c3b57c84844eef8ba4c0861a7dc9d9341bc8681
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Using cached pyro_ppl-1.9.1-py3-none-any.whl.metadata (7.8 kB)
  Using cached pyro_api-0.1.2-py3-none-any.whl.metadata (2.5 kB)
Using cached pyro_ppl-1.9.1-py3-none-any.whl (755 kB)
Using cached pyro_api-0.1.2-py3-none-any.whl (11 kB)
  Created wheel for pgmpy: filename=pgmpy-1.0.0-py3-none-any.whl size=2005365 sha256=6c41ed222944dc5d5ca249919ab2e708c9f35a0f67b5bf84ddb3d91264a87ff0
  Stored in directory: /private/var/folders/13/p9hvf8t929x0626wz_slh9340000gn/T/pip-ephem-wheel-ca

In [4]:
# Install required packages
!pip install pyitlib tensorflow matplotlib Pympler

  Using cached Pympler-1.1-py3-none-any.whl.metadata (3.6 kB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached Pympler-1.1-py3-none-any.whl (165 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.13.2
    Uninstalling typing_extensions-4.13.2:
      Successfully uninstalled typing_extensions-4.13.2


In [5]:
# Optional: Install causalnex for NOTEARS algorithm
!pip install causalnex

  Using cached pgmpy-0.1.19-py3-none-any.whl.metadata (6.3 kB)
Using cached pgmpy-0.1.19-py3-none-any.whl (1.9 MB)
  Attempting uninstall: pgmpy
    Found existing installation: pgmpy 1.0.0
    Uninstalling pgmpy-1.0.0:
      Successfully uninstalled pgmpy-1.0.0


In [6]:
# Optional: Install ucimlrepo for accessing UCI datasets
!pip install ucimlrepo

  Using cached ucimlrepo-0.0.7-py3-none-any.whl.metadata (5.5 kB)
Using cached ucimlrepo-0.0.7-py3-none-any.whl (8.0 kB)


In [ ]:
# Import packages and suppress warnings
import sys
import os
import warnings
import logging
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Suppress warnings
warnings.filterwarnings('ignore')
logging.getLogger('tensorflow').setLevel(logging.ERROR)
logging.getLogger('pgmpy').setLevel(logging.ERROR)

## Step 2: Install and Check RLiG

We need to ensure the RLiG package is properly installed. Since it's a local package, we'll need to install it from the local directory.

In [9]:
# Install RLiG locally
!cd ganblr-0.1.1 && pip install -e .

Obtaining file:///Users/harrietlesly/Mini-Project/RLiG/ganblr-0.1.1
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ganblr
    Found existing installation: ganblr 0.1.1
    Uninstalling ganblr-0.1.1:
      Successfully uninstalled ganblr-0.1.1
  DEPRECATION: Legacy editable install of ganblr==0.1.1 from file:///Users/harrietlesly/Mini-Project/RLiG/ganblr-0.1.1 (setup.py develop) is deprecated. pip 25.0 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running setup.py develop for ganblr


In [11]:
# Verify RLiG installation
try:
    from ganblr.models import RLiG
    print("RLiG successfully imported!")
except ImportError as e:
    print(f"Error importing RLiG: {e}")

Error importing RLiG: cannot import name 'AIC' from 'pgmpy.estimators' (/Users/harrietlesly/anaconda3/envs/rlig/lib/python3.8/site-packages/pgmpy/estimators/__init__.py)


## Step 3: Dataset Preparation

Let's prepare our datasets. We'll use a couple of small datasets first to verify everything works.

In [ ]:
# Define a function to get UCI datasets
def get_dataset(name, id=None, path=None):
    """
    Get a dataset by name, either from UCI repository or local path
    """
    if id is not None:
        try:
            from ucimlrepo import fetch_ucirepo
            dataset = fetch_ucirepo(id=id)
            X = dataset.data.features
            y = dataset.data.targets
            print(f"Loaded {name} dataset from UCI ML Repository")
            print(f"Features shape: {X.shape}, Target shape: {y.shape}")
            return X, y
        except Exception as e:
            print(f"Error loading UCI dataset: {e}")
    
    if path is not None:
        try:
            df = pd.read_csv(path)
            X = df.iloc[:, :-1]
            y = df.iloc[:, -1:]
            print(f"Loaded {name} dataset from {path}")
            print(f"Features shape: {X.shape}, Target shape: {y.shape}")
            return X, y
        except Exception as e:
            print(f"Error loading dataset from file: {e}")
    
    # If we get here, we couldn't load the dataset
    print(f"Could not load {name} dataset")
    return None, None

In [ ]:
# Test loading a dataset
X, y = get_dataset("Rice", id=545)
if X is not None:
    print("\nFeature columns:", X.columns.tolist())
    print("\nFirst few rows of X:")
    display(X.head())
    print("\nFirst few rows of y:")
    display(y.head())

## Step 4: Data Preprocessing

We need to preprocess our data to ensure it's in the right format for all models.

In [ ]:
# Import preprocessing packages
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, KBinsDiscretizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
def label_encode_cols(X, cols):
    """Label encode categorical columns"""
    X_encoded = X.copy()
    encoders = {}
    for col in cols:
        le = LabelEncoder()
        X_encoded[col] = le.fit_transform(X_encoded[col])
        encoders[col] = le
    return X_encoded, encoders

def preprocess_data(X, y):
    """Preprocess data: discretize continuous variables and encode categoricals"""
    # Identify column types
    continuous_cols = X.select_dtypes(include=['number']).columns
    categorical_cols = X.select_dtypes(include=['object']).columns
    
    print(f"Continuous columns: {len(continuous_cols)}")
    print(f"Categorical columns: {len(categorical_cols)}")
    
    # Create transformation pipeline
    transformers = []
    if len(continuous_cols) > 0:
        continuous_transformer = Pipeline(steps=[
            ('scaler', StandardScaler()),
            ('discretizer', KBinsDiscretizer(n_bins=5, encode='ordinal', strategy='uniform'))
        ])
        transformers.append(('num', continuous_transformer, continuous_cols))
    
    # Handle categorical columns
    if len(categorical_cols) > 0:
        X, encoders = label_encode_cols(X, categorical_cols)
    
    # Apply transformations
    preprocessor = ColumnTransformer(transformers=transformers, remainder='passthrough')
    X_transformed = preprocessor.fit_transform(X)
    X_transformed_df = pd.DataFrame(X_transformed, columns=continuous_cols.tolist() + categorical_cols.tolist())
    
    # Handle target variable
    if y.dtypes[0] == 'object':
        label_encoder = LabelEncoder()
        y_transformed = pd.DataFrame(label_encoder.fit_transform(y.values.ravel()), columns=y.columns)
    else:
        y_transformed = y
    
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(X_transformed_df, y_transformed, test_size=0.2, random_state=42, stratify=y)
    print(f"Training set: {X_train.shape[0]} samples")
    print(f"Test set: {X_test.shape[0]} samples")
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Test preprocessing on our dataset
if X is not None and y is not None:
    X_train, X_test, y_train, y_test = preprocess_data(X, y)
    
    print("\nPreprocessed training data (first few rows):")
    display(X_train.head())
    
    # Create combined dataset for pgmpy
    train_data = pd.concat([X_train, y_train], axis=1)
    print("\nCombined training data shape:", train_data.shape)
    display(train_data.head())

## Step 5: Testing the Baseline Models

Let's start by testing the baseline models one by one to make sure each works properly.

In [ ]:
# Import model packages
from pgmpy.estimators import HillClimbSearch, BicScore, TreeSearch, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.metrics import structure_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

In [ ]:
# Helper functions for model evaluation
def train_bn(model, data):
    """Train a Bayesian Network model"""
    bn = BayesianNetwork()
    bn.add_nodes_from(model.nodes())
    bn.add_edges_from(model.edges())
    
    # Fit model using Maximum Likelihood Estimation
    try:
        bn.fit(data, estimator=MaximumLikelihoodEstimator)
        return bn
    except Exception as e:
        print(f"Error fitting Bayesian Network: {e}")
        return None

def evaluate_bn_model(model, X_test, y_test):
    """Evaluate Bayesian Network classification performance"""
    if model is None:
        return None
    
    infer = VariableElimination(model)
    target_var = y_test.columns[0]  # Assumes only one target variable
    model_nodes = set(model.nodes())
    y_pred = []
    
    # Make predictions for each test instance
    for index, row in X_test.iterrows():
        evidence = {k: v for k, v in row.to_dict().items() if k in model_nodes}
        try:
            q = infer.map_query(variables=[target_var], evidence=evidence, show_progress=False)
            y_pred.append(q[target_var])
        except Exception as e:
            y_pred.append(None)
    
    # Filter out None predictions
    y_test_classes = y_test[target_var].unique()
    y_pred = [pred if pred in y_test_classes else None for pred in y_pred]
    
    valid_indices = [i for i, pred in enumerate(y_pred) if pred is not None]
    if not valid_indices:
        return 0.0  # No valid predictions
    
    y_pred = [y_pred[i] for i in valid_indices]
    y_test_filtered = y_test.iloc[valid_indices].values.ravel()
    
    # Calculate accuracy
    return accuracy_score(y_test_filtered, y_pred)

In [ ]:
# Test Hill Climbing Search
if 'train_data' in locals():
    print("Testing Hill Climbing Search...")
    try:
        hc = HillClimbSearch(train_data)
        best_model_hc = hc.estimate(scoring_method=BicScore(train_data))
        print(f"Hill Climbing Search model has {len(best_model_hc.nodes())} nodes and {len(best_model_hc.edges())} edges")
        print(f"Edges: {best_model_hc.edges()}")
        
        # Train the model
        bn_hc = train_bn(best_model_hc, train_data)
        
        # Evaluate
        if bn_hc:
            hc_acc = evaluate_bn_model(bn_hc, X_test, y_test)
            hc_bic = structure_score(bn_hc, train_data, scoring_method="bic")
            print(f"Hill Climbing - Accuracy: {hc_acc:.4f}, BIC: {hc_bic}")
    except Exception as e:
        print(f"Error with Hill Climbing Search: {e}")

In [ ]:
# Test Tree Search
if 'train_data' in locals():
    print("Testing Tree Search...")
    try:
        ts = TreeSearch(train_data)
        best_model_ts = ts.estimate()
        print(f"Tree Search model has {len(best_model_ts.nodes())} nodes and {len(best_model_ts.edges())} edges")
        print(f"Edges: {best_model_ts.edges()}")
        
        # Train the model
        bn_ts = train_bn(best_model_ts, train_data)
        
        # Evaluate
        if bn_ts:
            ts_acc = evaluate_bn_model(bn_ts, X_test, y_test)
            ts_bic = structure_score(bn_ts, train_data, scoring_method="bic")
            print(f"Tree Search - Accuracy: {ts_acc:.4f}, BIC: {ts_bic}")
    except Exception as e:
        print(f"Error with Tree Search: {e}")

In [ ]:
# Test Naive Bayes
if 'X_train' in locals() and 'y_train' in locals():
    print("Testing Naive Bayes...")
    try:
        nb = GaussianNB()
        nb.fit(X_train, y_train.values.ravel())
        
        # Evaluate
        y_pred = nb.predict(X_test)
        nb_acc = accuracy_score(y_test.values.ravel(), y_pred)
        print(f"Naive Bayes - Accuracy: {nb_acc:.4f}")
    except Exception as e:
        print(f"Error with Naive Bayes: {e}")

In [ ]:
# Test NOTEARS (if causalnex is available)
try:
    from causalnex.structure import StructureModel
    from causalnex.structure.notears import from_pandas
    
    if 'train_data' in locals():
        print("Testing NOTEARS...")
        try:
            sm = from_pandas(train_data, w_threshold=0.8)
            print(f"NOTEARS model has {len(sm.nodes)} nodes and {len(sm.edges)} edges")
            
            # Train the model
            bn_nt = train_bn(sm, train_data)
            
            # Evaluate
            if bn_nt:
                nt_acc = evaluate_bn_model(bn_nt, X_test, y_test)
                nt_bic = structure_score(bn_nt, train_data, scoring_method="bic")
                print(f"NOTEARS - Accuracy: {nt_acc:.4f}, BIC: {nt_bic}")
        except Exception as e:
            print(f"Error with NOTEARS: {e}")
except ImportError:
    print("CausalNex is not installed. NOTEARS will be skipped.")

## Step 6: Testing RLiG

Now let's test RLiG to ensure it works properly.

In [ ]:
# Test RLiG on original data format (not the preprocessed version)
if X is not None and y is not None:
    print("Testing RLiG...")
    try:
        from ganblr.models import RLiG
        
        # Initialize model
        rlig_model = RLiG()
        
        # Fit model - use small values for testing
        print("Fitting RLiG model (this may take a while)...")
        rlig_model.fit(X, y, episodes=2, gan=1, k=0, epochs=5, n=1)
        
        # Evaluate using RLiG's built-in evaluation
        print("Evaluating RLiG model...")
        lr_result = rlig_model.evaluate(X, y, model='lr')
        mlp_result = rlig_model.evaluate(X, y, model='mlp')
        rf_result = rlig_model.evaluate(X, y, model='rf')
        
        print(f"RLiG - LR Accuracy: {lr_result:.4f}")
        print(f"RLiG - MLP Accuracy: {mlp_result:.4f}")
        print(f"RLiG - RF Accuracy: {rf_result:.4f}")
        
        # Visualize the learned Bayesian network
        if hasattr(rlig_model, 'bayesian_network'):
            model_graphviz = rlig_model.bayesian_network.to_graphviz()
            model_graphviz.draw("rlig_network.png", prog="dot")
            print("RLiG network visualization saved to rlig_network.png")
            
            # Print network properties
            print(f"RLiG model has {len(rlig_model.bayesian_network.nodes())} nodes and {len(rlig_model.bayesian_network.edges())} edges")
            
            # BIC score
            if hasattr(rlig_model, 'best_score'):
                print(f"RLiG BIC score: {rlig_model.best_score}")
    except Exception as e:
        print(f"Error with RLiG: {e}")

## Step 7: Running the Full Comparison

Now that we've verified each model works individually, let's run the full comparison framework.

In [ ]:
# Import the full comparison framework
import sys
sys.path.append('../')  # Adjust this path as needed
from compare_models import compare_models, format_results

In [ ]:
# Define datasets to evaluate
# Start with small datasets for testing
datasets = {
    'Rice': 545,       # UCI ID for Rice dataset
    'TicTacToe': 101   # UCI ID for Tic-tac-toe dataset
}

In [ ]:
# Run the comparison
results = compare_models(datasets)

In [ ]:
# Format and display results
formatted_results = format_results(results)

print("\n\n=== ACCURACY RESULTS ===")
print(formatted_results['accuracy'])
print("\n\n=== TIME RESULTS (seconds) ===")
print(formatted_results['time'])
print("\n\n=== BIC SCORE RESULTS ===")
print(formatted_results['bic'])

In [ ]:
# Visualize the accuracy results
plt.figure(figsize=(12, 6))
ax = formatted_results['accuracy'].plot(kind='bar')
plt.title('Model Accuracy Comparison')
plt.ylabel('Accuracy')
plt.xlabel('Dataset')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='Model')

# Add value labels on the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.2f', padding=3)

plt.tight_layout()
plt.savefig('accuracy_comparison.png')
plt.show()

In [ ]:
# Visualize the time results
plt.figure(figsize=(12, 6))
ax = formatted_results['time'].plot(kind='bar')
plt.title('Model Training Time Comparison (seconds)')
plt.ylabel('Time (s)')
plt.xlabel('Dataset')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='Model')

# Add value labels on the bars
for container in ax.containers:
    ax.bar_label(container, fmt='%.1f', padding=3)

plt.tight_layout()
plt.savefig('time_comparison.png')
plt.show()

## Step 8: Troubleshooting

If you encounter issues, these debugging cells may help identify the problem.

In [ ]:
# Check RLiG details
import inspect
try:
    from ganblr.models import RLiG
    print("RLiG methods:")
    for method_name, method in inspect.getmembers(RLiG, predicate=inspect.isfunction):
        print(f"  - {method_name}")
except ImportError as e:
    print(f"Error importing RLiG: {e}")

In [ ]:
# Enhanced CPD normalization check with more detailed diagnostics
if 'rlig_model' in locals() and hasattr(rlig_model, 'bayesian_network'):
    print("Checking CPDs in RLiG model...")
    print(f"Total CPDs: {len(rlig_model.bayesian_network.get_cpds())}")
    
    # Check each CPD for normalization issues
    for cpd in rlig_model.bayesian_network.get_cpds():
        node = cpd.variable
        values = cpd.values
        sum_values = np.sum(values, axis=0)
        norm_condition = np.allclose(sum_values, 1.0, rtol=1e-5)
        
        print(f"\nNode: {node}")
        print(f"  - Values shape: {values.shape}")
        print(f"  - Sum to 1: {norm_condition}")
        
        if not norm_condition:
            # Display diagnostics for problematic CPDs
            print(f"  - Sum values: {sum_values}")
            print(f"  - Min value: {np.min(values)}, Max value: {np.max(values)}")
            print(f"  - Zeros count: {np.sum(values == 0)}/{values.size}")
            
            # Check for specific issues - find columns that don't sum to 1
            if values.ndim > 1:
                problematic_cols = np.where(~np.isclose(sum_values, 1.0, rtol=1e-5))[0]
                print(f"  - Problematic columns (indices): {problematic_cols}")
                
                if len(problematic_cols) > 0 and len(problematic_cols) < 10:
                    # Show sample values for a few problematic columns
                    for col_idx in problematic_cols[:3]:  # Show up to 3 examples
                        print(f"    Col {col_idx}: values = {values[:, col_idx]}, sum = {sum_values[col_idx]}")
                        
    # Check overall Bayesian Network consistency
    print("\nChecking Bayesian Network consistency...")
    try:
        is_valid = rlig_model.bayesian_network.check_model()
        print(f"Is valid model: {is_valid}")
    except Exception as e:
        print(f"Error checking model: {e}")
        
    # Try to detect the specific Minor_Axis_Length node issue
    if hasattr(rlig_model, 'variables'):
        print("\nSearching for potential 'Minor_Axis_Length' node...")
        for var in rlig_model.variables:
            if 'minor' in str(var).lower() or 'axis' in str(var).lower() or 'length' in str(var).lower():
                print(f"Found potential match: {var}")
                # Try to get the CPD for this node if it exists
                try:
                    cpd = rlig_model.bayesian_network.get_cpds(str(var))
                    if cpd:
                        print(f"CPD for {var}:")
                        print(f"- Shape: {cpd.values.shape}")
                        print(f"- Sums to 1: {np.allclose(np.sum(cpd.values, axis=0), 1.0)}")
                except Exception as e:
                    print(f"Error retrieving CPD for {var}: {e}")

## Step 9: Next Steps

Once you've confirmed everything works with these small datasets, you can expand to the full set of 13 datasets mentioned in the paper.

1. Add more datasets to the datasets dictionary
2. Run the full comparison
3. Analyze the results across all datasets

In [ ]:
# For full paper replication, use all 13 datasets
# This is just a template - adjust paths as needed
all_datasets = {
    'Adult': 2,
    'Rice': 545,
    'TicTacToe': 101,
    # Add other datasets here
    # 'Dataset_name': id_or_path
}

In [ ]:
# Optional: Run the full comparison (may take a long time)
# Uncomment to run
# full_results = compare_models(all_datasets)
# full_formatted_results = format_results(full_results)